### This notebook is used for tocknize sentences and process pre-trained GloVe embeddings

In [1]:
from __future__ import print_function

import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file

Using TensorFlow backend.


In [2]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 25

In [3]:
# load training data
question1 = []
question2 = []
is_duplicate = []
with open('./data/train.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        question1.append(row['question1'])
        question2.append(row['question2'])
        is_duplicate.append(row['is_duplicate'])
print('Question pairs: %d' % len(question1))

Question pairs: 404290


In [4]:
# load testing data
question1_test = []
question2_test = []

with open('./data/test.csv', encoding = 'utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        question1_test.append(row['question1'])
        question2_test.append(row['question2'])
print ('Question pairs in testing dataset: %d' % len(question1_test))

Question pairs in testing dataset: 2345796


In [5]:
questions = question1 + question2 + question1_test + question2_test
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)

question1_word_sequences = tokenizer.texts_to_sequences(question1)
question2_word_sequences = tokenizer.texts_to_sequences(question2)
question1_test_word_sequences = tokenizer.texts_to_sequences(question1_test)
question2_test_word_sequences = tokenizer.texts_to_sequences(question2_test)

word_index = tokenizer.word_index

print("Words in index: %d" % len(word_index))

Words in index: 137042


In [6]:
training_data = [question1_word_sequences, question2_word_sequences, is_duplicate]

In [7]:
len(training_data[1])

404290

In [8]:
testing_data = [question1_test_word_sequences, question2_test_word_sequences]

### save question sequence data here

In [9]:
# save question sequences
with open('./data/training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./data/testing_data.pickle', 'wb') as handle:
    pickle.dump(testing_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
# next time, load from here
with open('./data/training_data.pickle', 'rb') as handle:
    question1_word_sequences, question2_word_sequences, is_duplicate = pickle.load(handle)
    
with open('./data/testing_data.pickle', 'rb') as handle:
    question1_test_ws, question2_test_ws = pickle.load(handle)

### glove embedding with 100 dimension
avaiable from: https://nlp.stanford.edu/projects/glove/

In [11]:
embeddings_index = {}
with open('./data/glove/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 400000


In [12]:
nb_words = min(MAX_NB_WORDS, len(word_index))

word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 60874


In [14]:
# q1_data = pad_sequences(question1_word_sequences, maxlen = MAX_SEQUENCE_LENGTH)
# q2_data = pad_sequences(question2_word_sequences, maxlen = MAX_SEQUENCE_LENGTH)

# labels = np.array(is_duplicate, dtype = int)

# print('Shape of question1 data tensor:', q1_data.shape)
# print('Shape of question2 data tensor:', q2_data.shape)
# print('Shape of label tensor:', labels.shape)

In [13]:
word_embedding_matrix.shape

(137043, 100)

In [15]:
# save question sequences
with open('./data/word_embedding_matrix_glove100.pickle', 'wb') as handle:
    pickle.dump(word_embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

### glove embedding with 300 dimension

In [9]:
embeddings_index = {}
with open('./data/glove/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 400000


In [11]:
EMBEDDING_DIM = 300

nb_words = min(MAX_NB_WORDS, len(word_index))

word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 60874


In [12]:
word_embedding_matrix.shape

(137043, 300)

In [13]:
# save question sequences
with open('./data/word_embedding_matrix_glove300.pickle', 'wb') as handle:
    pickle.dump(word_embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Reference: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html